In [138]:
database='D:/Cryptography/Deeplearning/NLP/material/wikiextractor/extracted_xml/AA/wiki_02'##怎么将文件都整合在一起

In [139]:
import os

In [140]:
os.path.exists(database)

True

In [141]:
import pandas as pd

In [142]:
dataframe= open(database, 'r',encoding='UTF-8')

PermissionError: [Errno 13] Permission denied: 'D:/Cryptography/Deeplearning/NLP/material/wikiextractor/extracted_xml/test'

In [50]:
DATA=dataframe.read()

In [143]:
DATA

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [144]:
len(DATA)

1290467

In [145]:
from hanziconv import HanziConv

In [57]:
data=HanziConv.toSimplified(DATA)

In [58]:
data

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [59]:
import re

In [60]:
def token(string):
    return ' '.join(re.findall('[\w|\d]+',string))

In [110]:
all_articles=token(data)

In [111]:
import jieba

In [112]:
def cut(string): return list(jieba.cut(string))

In [113]:
ALL_TOKENS = cut(all_articles)

In [114]:
valida_tokens = [t for t in ALL_TOKENS if t.strip() and t!='n']

In [115]:
len(ALL_TOKENS)

703457

In [116]:
len(valida_tokens)

580080

In [117]:
from collections import Counter

In [118]:
words_count = Counter(valida_tokens)

In [119]:
words_count.most_common(10)

[('的', 30997),
 ('在', 8143),
 ('年', 7760),
 ('和', 6220),
 ('是', 5788),
 ('为', 4230),
 ('有', 3602),
 ('了', 3434),
 ('与', 3215),
 ('中国', 3185)]

In [120]:
frequences_all = [f for w, f in words_count.most_common()]

In [121]:
frequences_sum = sum(frequences_all)

In [122]:
frequences_sum

580080

In [123]:
1 / frequences_sum

1.7239001517032133e-06

In [124]:
def get_prob(word):
    esp = 1/ frequences_sum
    if word in words_count:
        return words_count[word] / frequences_sum
    else:
        return esp

In [125]:
get_prob('青岛')

7.75755068266446e-05

In [126]:
valid_tokens = [str(t) for t in valida_tokens]

In [127]:
all_2_grams_words = [''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]

In [128]:
_2_gram_sum = len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)

def get_combination_prob(w1, w2):
    if w1+ w2 in _2_gram_counter: return _2_gram_counter[w1 + w2] / _2_gram_sum
    else:
        return 1 / _2_gram_sum

In [129]:
get_combination_prob('去','北京')

1.723906095387172e-06

In [130]:
def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

In [131]:
def language_model_of_2_gram(sentences):
    sentence_probability = 1
    
    words = cut(sentences)
    
    
    for i, word in enumerate(words):
        if i== 0:
            prob = get_prob(word)
        else:
            previous = words[i-1]
            prob = get_prob_2_gram(previous, word)
        
        sentence_probability *= prob
    
    return sentence_probability
        

In [132]:
language_model_of_2_gram('小明今天抽奖抽到一台苹果手机')

1.0990283144934068e-10

In [133]:
language_model_of_2_gram('小明今天抽奖抽到一台波音飞机')

8.425883744449452e-10

In [134]:
need_compared= [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一直好看的小猫 真是一只好看的小猫",
    "今晚火锅去吃我 今晚我去吃火锅"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = language_model_of_2_gram(s1),language_model_of_2_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} wirh probility {}'.format(s1,p1))
    print('-'*4 + ' {} wirh probility {}'.format(s2,p2))    

明天晚上请你吃大餐，我们一起吃苹果 is more possible
---- 今天晚上请你吃大餐，我们一起吃日料 wirh probility 1.1342624881147892e-16
---- 明天晚上请你吃大餐，我们一起吃苹果 wirh probility 1.1342624881147892e-16
真是一只好看的小猫 is more possible
---- 真事一直好看的小猫 wirh probility 2.4530708880606703e-15
---- 真是一只好看的小猫 wirh probility 3.972559387226425e-12
今晚我去吃火锅 is more possible
---- 今晚火锅去吃我 wirh probility 1.829097004309819e-10
---- 今晚我去吃火锅 wirh probility 1.9490310715845879e-10


## 吃大餐这里又不对了，是语料太少的原因吗